<a href="https://colab.research.google.com/github/EAbdelrahman/Player-Performance-Predictor/blob/main/Rate_Model(best).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, precision_score, \
recall_score, cohen_kappa_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 999
pd.options.display.max_rows = 50 

# importing data 

In [4]:
from google.colab import files
uploaded= files.upload()

Saving all_Data_all_the_time.xlsx to all_Data_all_the_time.xlsx


In [5]:
import io
Full_Data = pd.read_excel(io.BytesIO(uploaded['all_Data_all_the_time.xlsx']))

# Handling Data

##Clean the Data

In [35]:
filtered_by_league=Full_Data[Full_Data.league=='Premier League']
filtered_by_league_NULL=filtered_by_league[filtered_by_league.rating>0]
filtered_by_league_NULL_FW=filtered_by_league_NULL[filtered_by_league_NULL.position=='Attacker']



## Split Data into Target and Features

## Feature Vector

Let's create a feature vector.

In [ ]:
#list(filtered_by_league_NULL_FW.columns)
filtered_by_league_NULL_FW.columns


Index(['player_id', 'player_name', 'firstname', 'lastname', 'number',
       'position', 'age', 'birth_date', 'birth_place', 'birth_country',
       'nationality', 'height', 'weight', 'injured', 'rating', 'team_id',
       'team_name', 'league_id', 'league', 'season', 'captain', 'shots/total',
       'shots/on', 'goals/total', 'goals/conceded', 'goals/assists',
       'goals/saves', 'passes/total', 'passes/key', 'passes/accuracy',
       'tackles/total', 'tackles/blocks', 'tackles/interceptions',
       'duels/total', 'duels/won', 'dribbles/attempts', 'dribbles/success',
       'fouls/drawn', 'fouls/committed', 'cards/yellow', 'cards/yellowred',
       'cards/red', 'penalty/won', 'penalty/commited', 'penalty/success',
       'penalty/missed', 'penalty/saved', 'games/appearences',
       'games/minutes_played', 'games/lineups', 'substitutes/in',
       'substitutes/out', 'substitutes/bench'],
      dtype='object')

In [79]:
features = ['captain','shots/total'
,'shots/on','goals/total','goals/assists','passes/total','passes/key','passes/accuracy','tackles/total','tackles/blocks'
,'tackles/interceptions','duels/total','duels/won','dribbles/attempts','dribbles/success','fouls/drawn','fouls/committed','cards/yellow','cards/yellowred','cards/red'
,'penalty/won','penalty/commited','penalty/success','penalty/missed','games/appearences','games/minutes_played','games/lineups','substitutes/in','substitutes/out'
,'substitutes/bench','results','players__player_id','players__player_name','players__firstname','players__lastname','players__number','players__position'
,'players__age','players__birth_date','players__birth_place','players__birth_country','players__nationality','players__height','players__weight','players__injured'
,'players__rating','players__team_id','players__team_name','players__league_id','players__league','players__season','players__captain','players__shots__total'
,'players__shots__on','players__goals__total','players__goals__conceded','players__goals__assists','players__goals__saves','players__passes__total','players__passes__key'
,'players__passes__accuracy','players__tackles__total','players__tackles__blocks','players__tackles__interceptions','players__duels__total','players__duels__won'
,'players__dribbles__attempts','players__dribbles__success','players__fouls__drawn','players__fouls__committed','players__cards__yellow','players__cards__yellowred'
,'players__cards__red','players__penalty__won','players__penalty__commited','players__penalty__success','players__penalty__missed','players__penalty__saved'
,'players__games__appearences','players__games__minutes_played','players__games__lineups','players__substitutes__in'
,'players__substitutes__out','players__substitutes__bench']


x=filtered_by_league_NULL_FW.iloc[:,20:53]
X=x.drop(columns=['goals/conceded','goals/saves','penalty/saved'])

#Y=(filtered_by_league_NULL_FW['rating']*100).astype('int') #For Classification

Y=filtered_by_league_NULL_FW['rating']

X.columns


Index(['captain', 'shots/total', 'shots/on', 'goals/total', 'goals/assists',
       'passes/total', 'passes/key', 'passes/accuracy', 'tackles/total',
       'tackles/blocks', 'tackles/interceptions', 'duels/total', 'duels/won',
       'dribbles/attempts', 'dribbles/success', 'fouls/drawn',
       'fouls/committed', 'cards/yellow', 'cards/yellowred', 'cards/red',
       'penalty/won', 'penalty/commited', 'penalty/success', 'penalty/missed',
       'games/appearences', 'games/minutes_played', 'games/lineups',
       'substitutes/in', 'substitutes/out', 'substitutes/bench'],
      dtype='object')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [7]:
#for vesion only
filtered_by_league_NULL_FW[['rating']].describe()

,rating
count,465.000000
mean,6.810627
std,0.288005
min,5.800000
25%,6.633333
50%,6.771875
75%,6.946666
max,7.856756


### Scaling the feature vector

Let's scale the feature vector.

First the standard scaler substracts by the mean and divides by std.  Let's try that ourselves and see if that looks right.

In [44]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
#y_train = sc_y.fit_transform(y_train)

### Reducing Dimensions( **Whitening Data**)



In [25]:
#for vesion only
X.corr().head(2)

,captain,shots/total,shots/on,goals/total,goals/assists,passes/total,passes/key,passes/accuracy,tackles/total,tackles/blocks,tackles/interceptions,duels/total,duels/won,dribbles/attempts,dribbles/success,fouls/drawn,fouls/committed,cards/yellow,cards/yellowred,cards/red,penalty/won,penalty/commited,penalty/success,penalty/missed,games/appearences,games/minutes_played,games/lineups,substitutes/in,substitutes/out,substitutes/bench
captain,1.000000,0.126171,0.140966,0.159797,0.037557,0.096367,0.099655,-0.063485,0.071584,0.169835,0.064267,0.191938,0.174124,-0.031664,-0.042654,0.072215,0.135878,0.117818,-0.016583,0.319619,-0.010213,0.085788,0.314003,0.278546,0.123194,0.166197,0.163650,-0.074054,0.027628,-0.096654
shots/total,0.126171,1.000000,0.971949,0.891769,0.692244,0.786137,0.782522,0.158564,0.569470,0.471094,0.525303,0.764940,0.724605,0.691193,0.646313,0.674270,0.679163,0.549135,0.087159,0.145427,0.423871,0.117054,0.493960,0.297705,0.804802,0.891204,0.888037,-0.097704,0.671683,-0.275060


###Because the stats are **Very heavily Correlated**

May Lead ** Biasing ! **
*  Reduces training time — due to smaller dataset
*  Removes noise — by keeping only what’s relevant


In [45]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_r = pca.fit(X_train).transform(X_train)

**The components are the two axes of the projected back into the original features space.
We can use these to calculate the relative components of the original features in our 
new features.**


In [46]:
# Let's get the top components of PC1:
print("top components of Principal Component 1(Eigenvector):")

#SS (Sum 0f square distances)
SS_Distances_PC1 = np.abs(pca.components_[0])/np.sum(np.abs(pca.components_[0])) # rel_values
print("Feature Names: " + str([features[i] for i in np.argsort(-SS_Distances_PC1)[:5]]))
print("Percentages: " + str(SS_Distances_PC1[np.argsort(-SS_Distances_PC1)[:5]]))

top components of Principal Component 1(Eigenvector):
Feature Names: ['games/minutes_played', 'games/lineups', 'duels/total', 'shots/total', 'passes/total']
Percentages: [0.05168771 0.05161854 0.04799575 0.04759392 0.04756509]


In [47]:
# Let's get the top components of PC2:
print("top components of Principal Component 2 :")

SS_Distances_PC2 = np.abs(pca.components_[1])/np.sum(np.abs(pca.components_[1]))
print("Feature Names: " + str([features[i] for i in np.argsort(-SS_Distances_PC2)[:3]]))
print("Percentages: " + str(SS_Distances_PC2[np.argsort(-SS_Distances_PC2)[:3]]))

top components of Principal Component 2 :
Feature Names: ['substitutes/in', 'substitutes/bench', 'penalty/success']
Percentages: [0.13001784 0.12256779 0.08877636]



# Linear Regression in Sklearn 

### Overview
Instructor to demo this on screen.
 
### Builds on
None

### Run time
approx. 20-30 minutes

In [31]:
import statsmodels.formula.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
# Create linear regression object
lin_reg_mod = linear_model.LinearRegression()
lin_reg_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
y_pred = lin_reg_mod.predict(X_test)
#filtered_by_league_NULL_FW['New_Rate']=lin_reg_mod.predict(X)

#Evaluation Of The Model


We'll now check the predictions against the actual values by using the RMSE and R-2 metrics, two metrics commonly used to evaluate regression tasks:

In [69]:
Result = pd.DataFrame({'Rate':y_test ,'Predicted_Rate':y_pred})
Result

,Rate,Predicted_Rate
4292,6.710810,6.696621
4602,7.140625,7.226573
6160,6.946666,6.829325
4699,6.809375,6.784527
1374,6.512500,6.595281
...,...,...
1198,7.206250,7.345957
6720,6.806250,6.865923
1572,6.562500,6.564732
7407,6.507692,6.647211


In [66]:
test_set_rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

test_set_r2 = r2_score(y_test, y_pred)

In [71]:
print('The Error percentage(RMSE) in our model = {}'.format(test_set_rmse))
#print(test_set_r2)

The Error percentage(RMSE) in our model = 0.13187428077439212


In [73]:
!pip install eli5

     |████████████████████████████████| 112kB 7.4MB/s 


In [80]:
import eli5
eli5.explain_weights(lin_reg_mod, feature_names=list(X.columns))

Weight?,Feature
+6.823,<BIAS>
+0.211,duels/won
+0.169,passes/key
+0.154,dribbles/success
+0.105,goals/total
+0.084,shots/total
+0.025,tackles/blocks
+0.022,shots/on
+0.013,goals/assists
+0.010,tackles/total


In [60]:
# Building the optimal model using Backward Elimination
import statsmodels.api as sm
X = np.append(arr = np.ones((465, 1)).astype(int), values = X, axis = 1)
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit() #ordinary least
regressor_OLS.summary()
X_opt = X[:, [0, 1, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1657: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ess/self.df_model
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1657: RuntimeWarning: invalid value encountered in double_scalars
  return self.ess/self.df_model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 21 Jan 2021   Prob (F-statistic):                nan
Time:                        12:26:55   Log-Likelihood:                -80.484
No. Observations:                 465   AIC:                             163.0
Df Residuals:                     464   BIC:                             167.1
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.4053      0.007    509.934      0.000       3.392       3.418
x1             3.4053      0.007    509.934      0.000       3.392       3.418
==============================================================================
Omnibus:                       63.398   Durbin-Watson:                   1.581
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              100.018
Skew:                           0.866   Prob(JB):                     1.91e-22
Kurtosis:                       4.471   Cond. No.                     2.28e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.79e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""